Testing multiple choices for lambda_pen on ER([500, 600, ..., 1000], [0.3, 0.5, 0.7], seed = 0 to 49)

In [ ]:
import torch
import networkx as nx
import numpy as np
import time

"""
Extended experiments: loop over graph sizes, edge probabilities, and lambda penalties.
- Graph sizes: 500, 600, 700, 800, 900, 1000
- Edge probabilities: 0.3, 0.5, 0.7
- Lambda_pen values: 0.0, 0.1, 0.2, ..., 1.0
- Seeds: 0 to 49 (50 instances per combination)
- Measure per seed: runs, mean/min/max MIS size, runtime
- Compute average runtime across seeds for each config
"""

# Experiment parameters
graph_sizes = list(range(900, 1001, 100)) # Output 一次只能做兩個100，之前有700 800，現在做500 600，之後再做900 1000
ps          = [0.3, 0.5, 0.7]
lambda_vals = [i/10 for i in range(11)]
seeds       = range(50)

# Algorithm parameters
beta      = 0.8
alpha     = 0.0001
ITER_T    = 150
NUM_INITS = 50
eta       = 2.0

# Parameters
gamma_c = 1

# Initialization penalty decay
decay_rate = 1
BATCH_SIZE = 10

# Efficient MIS checker
def MIS_checker_efficient_3(X, A, Abar):
    N = X.shape[0]
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones(N) + (N * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

# Degree-based init vector
def make_degree_init(G):
    deg = dict(G.degree())
    max_deg_node = max(deg.values())
    d = torch.tensor([1 - deg[i] / max_deg_node for i in range(len(deg))], dtype=torch.float32)
    return d / d.max()

# Run single seed experiment
def run_seed_experiment(n, p, seed, lambda_pen):
    # Build graph
    G = nx.fast_gnp_random_graph(n, p, seed=seed)
    comp = nx.complement(G)
    max_deg = max(dict(comp.degree()).values())
    gamma = 2 + max_deg

    A    = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.float32)
    Abar = torch.tensor(nx.adjacency_matrix(comp).todense(), dtype=torch.float32)
    d    = make_degree_init(G)
    cov  = eta * torch.eye(n)

    penalty_count = np.zeros(n, dtype=int)
    found_sizes = []

    start_time = time.time()
    for init in range(NUM_INITS):
        bias   = 1.0 / (1 + lambda_pen * penalty_count)
        bias_t = torch.from_numpy(bias).float()
        if init == 0:
            x = d.clone() * bias_t
        else:
            torch.manual_seed(init)
            base = torch.normal(mean=d, std=torch.sqrt(torch.diag(cov)))
            x = (base * bias_t).clamp(0, 1)
        v = torch.zeros(n)

        done = False
        for _ in range(ITER_T):
            grad = -torch.ones(n) + (gamma * A - gamma_c * Abar) @ x
            v    = beta * v + alpha * grad
            x    = (x - v).clamp(0, 1)
            done, MIS = MIS_checker_efficient_3(x, A, Abar)
            if done:
                break

        if done:
            found_sizes.append(len(MIS))
            penalty_count[MIS.tolist()] += 1

        if init > 0 and init % BATCH_SIZE == 0:
            penalty_count = np.maximum(penalty_count - decay_rate, 0)
    runtime = time.time() - start_time

    if found_sizes:
        return len(found_sizes), np.mean(found_sizes), np.min(found_sizes), np.max(found_sizes), runtime
    else:
        return 0, 0.0, 0, 0, runtime

# Main experimental loop
for n in graph_sizes:
    for p in ps:
        print(f"\n=== Graph ER({n}, {p}) ===")
        for lam in lambda_vals:
            seed_results = []
            for seed in seeds:
                runs, mean_s, min_s, max_s, rt = run_seed_experiment(n, p, seed, lam)
                seed_results.append((seed, runs, mean_s, min_s, max_s, rt))
                print(f"λ={lam:.1f} Seed {seed:2d}: runs={runs}, mean={mean_s:.2f}, min={min_s}, max={max_s}, time={rt:.1f}s")
            avg_rt = np.mean([r[5] for r in seed_results])
            print(f"  → λ={lam:.1f} avg runtime per seed: {avg_rt:.1f}s")



=== Graph ER(900, 0.3) ===
λ=0.0 Seed  0: runs=50, mean=18.32, min=15, max=22, time=8.7s
λ=0.0 Seed  1: runs=50, mean=18.60, min=15, max=21, time=10.5s
λ=0.0 Seed  2: runs=50, mean=18.66, min=15, max=22, time=11.7s
λ=0.0 Seed  3: runs=50, mean=18.58, min=16, max=21, time=11.0s
λ=0.0 Seed  4: runs=50, mean=18.68, min=16, max=21, time=10.3s
λ=0.0 Seed  5: runs=50, mean=18.30, min=16, max=21, time=10.5s
λ=0.0 Seed  6: runs=50, mean=18.78, min=15, max=22, time=30.5s
λ=0.0 Seed  7: runs=50, mean=18.24, min=16, max=20, time=11.5s
λ=0.0 Seed  8: runs=50, mean=18.16, min=16, max=21, time=27.0s
λ=0.0 Seed  9: runs=50, mean=18.12, min=16, max=20, time=12.0s
λ=0.0 Seed 10: runs=50, mean=18.38, min=15, max=21, time=11.3s
λ=0.0 Seed 11: runs=50, mean=18.40, min=16, max=22, time=10.9s
λ=0.0 Seed 12: runs=50, mean=18.26, min=16, max=21, time=8.8s
λ=0.0 Seed 13: runs=50, mean=18.26, min=16, max=21, time=9.9s
λ=0.0 Seed 14: runs=50, mean=18.44, min=16, max=21, time=10.0s
λ=0.0 Seed 15: runs=50, mean=1

Decreasing lambda_pen schedule testing on ER(800, 0.5, seed 0 to 49)

In [ ]:
import torch
import networkx as nx
import numpy as np

'''
Multi-seed MIS runs with a decreasing initialization penalty schedule:
lambda_pen = 1.0 - 0.1 * (init // 5)
- Loops over graph seeds 0 to 49.
- For each graph, runs NUM_INITS restarts with degree-based initialization bias.
- Reports summary statistics for each seed.
'''

# Graph and algorithm parameters
n = 800
p = 0.5
gamma_c   = 1
beta      = 0.8
alpha     = 0.0001
ITER_T    = 150
NUM_INITS = 50
eta       = 2.0  # exploration noise scale

# Decay parameters
decay_rate = 1
BATCH_SIZE = 10

def make_degree_init(G):
    deg = dict(G.degree())
    max_deg_node = max(deg.values())
    d = torch.tensor([1 - deg[i] / max_deg_node for i in range(len(deg))],
                     dtype=torch.float32)
    return d / d.max()

def MIS_checker_efficient_3(X, A, Abar):
    n = X.shape[0]
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones(n) + (n * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def run_on_seed(seed):
    # build graph and its complement
    G     = nx.gnp_random_graph(n, p, seed=seed)
    Gbar  = nx.complement(G)

    # adjacency tensors
    A     = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.float32)
    Abar  = torch.tensor(nx.adjacency_matrix(Gbar).todense(),
                         dtype=torch.float32)

    # compute gamma = 2 + max degree in complement graph
    max_deg_bar = max(dict(Gbar.degree()).values())
    gamma       = 2 + max_deg_bar

    # degree‐based init vector
    d            = make_degree_init(G)
    cov_matrix   = eta * torch.eye(n)

    penalty_count = np.zeros(n, dtype=int)
    found_sizes   = []

    for init in range(NUM_INITS):
        # decreasing lambda_pen schedule
        lambda_pen = max(0.0, 1.0 - 0.1 * (init // 5))

        # initialization bias
        bias   = 1.0 / (1 + lambda_pen * penalty_count)
        bias_t = torch.from_numpy(bias).float()

        if init == 0:
            x = d.clone() * bias_t
        else:
            torch.manual_seed(init)
            base = torch.normal(mean=d, std=torch.sqrt(torch.diag(cov_matrix)))
            x    = base * bias_t

        v    = torch.zeros(n)
        done = False

        # gradient‐descent loop
        for _ in range(ITER_T):
            grad = -torch.ones(n) + (gamma * A - gamma_c * Abar) @ x
            v    = beta * v + alpha * grad
            x    = (x - v).clamp(0, 1)
            done, MIS = MIS_checker_efficient_3(x, A, Abar)
            if done:
                break

        if done:
            nodes = MIS.tolist()
            found_sizes.append(len(nodes))
            penalty_count[nodes] += 1

        # decay penalty counters every BATCH_SIZE inits
        if init > 0 and init % BATCH_SIZE == 0:
            penalty_count = np.maximum(penalty_count - decay_rate, 0)

    # print summary
    if found_sizes:
        mean_size = np.mean(found_sizes)
        min_size  = np.min(found_sizes)
        max_size  = np.max(found_sizes)
    else:
        mean_size = min_size = max_size = 0

    print(f"Seed {seed:2d}: runs={len(found_sizes)}, "
          f"mean={mean_size:.2f}, min={min_size}, max={max_size}")

# Main loop
print(f"Running MIS with decreasing init penalty schedule "
      f"and gamma from complement graph max‐degree:")
for seed in range(50):
    run_on_seed(seed)




Running MIS with decreasing init penalty schedule and gamma from complement graph max‐degree:
Seed  0: runs=50, mean=10.18, min=8, max=13
Seed  1: runs=50, mean=10.36, min=8, max=13
Seed  2: runs=50, mean=9.92, min=8, max=11
Seed  3: runs=50, mean=9.98, min=7, max=12
Seed  4: runs=50, mean=10.20, min=8, max=12
Seed  5: runs=50, mean=10.14, min=8, max=12
Seed  6: runs=50, mean=9.94, min=8, max=11
Seed  7: runs=50, mean=10.08, min=9, max=13
Seed  8: runs=50, mean=9.94, min=8, max=12
Seed  9: runs=50, mean=10.36, min=9, max=12
Seed 10: runs=50, mean=9.94, min=8, max=12
Seed 11: runs=50, mean=10.18, min=9, max=12
Seed 12: runs=50, mean=10.00, min=8, max=12
Seed 13: runs=50, mean=10.16, min=8, max=12
Seed 14: runs=50, mean=10.26, min=8, max=13
Seed 15: runs=50, mean=9.84, min=8, max=12
Seed 16: runs=50, mean=10.02, min=8, max=11
Seed 17: runs=50, mean=10.04, min=8, max=12
Seed 18: runs=50, mean=10.34, min=9, max=13
Seed 19: runs=50, mean=10.24, min=8, max=13
Seed 20: runs=50, mean=10.26, mi

In [ ]:
import torch
import networkx as nx
import numpy as np
import pandas as pd

# Algorithm parameters (common)
gamma_c   = 1
beta      = 0.8
alpha     = 0.0001
ITER_T    = 150
NUM_INITS = 50
eta       = 2.0

decay_rate = 1
BATCH_SIZE = 10

# Graph categories to test: (n, p)
graph_params = [
    (500, 0.3),
    (500, 0.7),
    (800, 0.3),
    (800, 0.7),
]

def make_degree_init(G):
    deg = dict(G.degree())
    max_deg_node = max(deg.values())
    d = torch.tensor([1 - deg[i] / max_deg_node for i in range(len(deg))],
                     dtype=torch.float32)
    return d / d.max()

def MIS_checker_efficient_3(X, A, Abar):
    n = X.shape[0]
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones(n) + (n * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def run_variant(n, p, seed, use_schedule):
    # build G and Gbar
    G    = nx.gnp_random_graph(n, p, seed=seed)
    Gbar = nx.complement(G)

    A    = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.float32)
    Abar = torch.tensor(nx.adjacency_matrix(Gbar).todense(),
                        dtype=torch.float32)

    # gamma = 2 + max_deg(Gbar)
    max_deg_bar = max(dict(Gbar.degree()).values())
    gamma       = 2 + max_deg_bar

    d            = make_degree_init(G)
    cov_matrix   = eta * torch.eye(n)
    penalty_count = np.zeros(n, dtype=int)
    found_sizes   = []

    for init in range(NUM_INITS):
        # dynamic lambda schedule vs. no penalty
        if use_schedule:
            lambda_pen = max(0.0, 1.0 - 0.1 * (init // 5))
        else:
            lambda_pen = 0.0

        bias   = 1.0 / (1 + lambda_pen * penalty_count)
        bias_t = torch.from_numpy(bias).float()

        if init == 0:
            x = d.clone() * bias_t
        else:
            torch.manual_seed(init)
            base = torch.normal(mean=d,
                                std=torch.sqrt(torch.diag(cov_matrix)))
            x    = base * bias_t

        v, done = torch.zeros(n), False
        for _ in range(ITER_T):
            grad = -torch.ones(n) + (gamma * A - gamma_c * Abar) @ x
            v    = beta * v + alpha * grad
            x    = (x - v).clamp(0, 1)
            done, MIS = MIS_checker_efficient_3(x, A, Abar)
            if done:
                break

        if done:
            nodes = MIS.tolist()
            found_sizes.append(len(nodes))
            penalty_count[nodes] += 1

        if init > 0 and init % BATCH_SIZE == 0:
            penalty_count = np.maximum(penalty_count - decay_rate, 0)

    return np.array(found_sizes)

# Run comparisons across all graph classes:
results = []
for (n, p) in graph_params:
    # track per-seed outcomes
    seed_outcomes = []
    for seed in range(50):
        sched_sizes = run_variant(n, p, seed, use_schedule=True)
        nopen_sizes = run_variant(n, p, seed, use_schedule=False)

        best_sched = sched_sizes.max() if sched_sizes.size else 0
        best_nopen = nopen_sizes.max() if nopen_sizes.size else 0

        if best_sched > best_nopen:
            outcome = 'schedule win'
        elif best_sched < best_nopen:
            outcome = 'no_penalty win'
        else:
            outcome = 'tie'
        seed_outcomes.append(outcome)

    # summarize
    counts = pd.Series(seed_outcomes).value_counts()
    results.append({
        'n':               n,
        'p':               p,
        'schedule wins':   int(counts.get('schedule win', 0)),
        'no_penalty wins': int(counts.get('no_penalty win', 0)),
        'ties':            int(counts.get('tie', 0)),
    })

# display as DataFrame
df_summary = pd.DataFrame(results)
print(df_summary.to_string(index=False))
